In [29]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install html

  Using cached https://files.pythonhosted.org/packages/4a/df/0e3d22d50ee43274eb5116f49972a164d853bb3ab305a69a0540b6292252/html-1.16.tar.gz
    ERROR: Complete output from command python setup.py egg_info:
    ERROR: Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/setuptools/__init__.py", line 20, in <module>
        from setuptools.depends import Require
      File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/setuptools/depends.py", line 6, in <module>
        from .py33compat import Bytecode
      File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/setuptools/py33compat.py", line 11, in <module>
        from setuptools.extern.six.moves import html_parser
      File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/setuptools/_vendor/six.py", line 92, in __get__
        result = self._resolve()
      File "/home/jupyterlab/conda/envs

In [37]:
import pandas as pd
import numpy as np
import folium
import requests
import matplotlib
import lxml.html as lh
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

matplotlib.style.use('ggplot')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [38]:
#Getting the major financial cities in the world from Wikipedia rankings based on the Global Financial Centre Index
df=pd.read_html('https://en.wikipedia.org/wiki/Global_Financial_Centres_Index', skiprows=1, header=0)
df=pd.DataFrame(df[0])
column_list=['Rank', 'Rank Change', 'Centre', 'GFCI Rating', 'Rating Change'] #Sanitizing the column list
df.columns=column_list
df['Rank Change'].replace(to_replace=np.nan, value=0, inplace=True)
df.drop(index=54, inplace=True) # This dropping the duplicated heading that came from the Wikipedia page
df.head()

,Rank,Rank Change,Centre,GFCI Rating,Rating Change
0,1,0,New York City,769,21
1,2,0,London,742,31
2,3,3,Tokyo,741,16
3,4,1,Shanghai,740,21
4,5,1,Singapore,738,24


In [39]:
# Removing 'City' to enable an accurate left join
city_list_org=[]
for i in df['Centre']:
        if 'City' in i:
            if 'Gujarat' in i:
                i=str(i.split(' City')[1])
                city_list_org.append(i)
            else:
                i=str(i.split(' City')[0])
                city_list_org.append(i)
        elif ' (' in i:
            i=str(i.split(' (')[0])
            city_list_org.append(i)
            
        else:
            city_list_org.append(i)

df['Centre']=city_list_org

In [40]:
#Using Geocode to get coordinate data for the list of cities

lat_list=[]
lng_list=[]
for city in df['Centre']:
    address=city
    geolocator=Nominatim(user_agent='foursquare_agent', timeout=10)
    location=geolocator.geocode(address)
    lat_list.append(location.latitude)
    lng_list.append(location.longitude)
#     print(address) This code is there to check any errors in retrieving data from Nominatim
    
df['Latitude']=lat_list
df['Longitude']=lng_list

df.head()

,Rank,Rank Change,Centre,GFCI Rating,Rating Change,Latitude,Longitude
0,1,0,New York,769,21,40.712728,-74.006015
1,2,0,London,742,31,51.507322,-0.127647
2,3,3,Tokyo,741,16,35.682839,139.759455
3,4,1,Shanghai,740,21,31.225299,121.489050
4,5,1,Singapore,738,24,1.357107,103.819499


In [44]:
### Sample check on the coordinates on the world map

world_map=folium.Map([28.614179, 77.202266], zoom_start=1.5) #Just setting the coords to New Delhi to get a zoom level for the map

for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['Centre']):
    label=f'{city}'
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng], 
                        radius=5, 
                        popup=label, 
                        color='blue', 
                        fill=True, 
                        fill_color='yellow', 
                        fill_opacity=0.7, parse_html=False).add_to(world_map)
world_map

In [45]:
CLIENT_ID='VEZTE0WLFZPUP01YMZWMUCULBOX0RRPRQGFDMD2PYXYWJRNC'
CLIENT_SECRET='GYLM3IBRBHDFXA5RTNBR4PUYBRMCVX5LOMBSN1B2POAARBPN'
VERSION='20180605'
Radius=10000 #Some cities does not have enough places, so I have arbitrarily for this purpose set radius to 10000 to get
                #a location just so that I can get the country name
Limit=2 #Just need one venue to identify the corresponding country for the given city

In [7]:
country_list=[]
for lat, lng, centre in zip(df['Latitude'], df['Longitude'], df['Centre']):
    url=f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={lat},{lng}&v={VERSION}&radius={Radius}&limit={Limit}&locale=en'
    result=requests.get(url).json()
    try:
        country=pd.json_normalize(result['response']['groups'][0]['items'])['venue.location.country'][0]
#         print(country, centre) error check code
        country_list.append(country)
    except (KeyError, ValueError):
#         print(centre) error check code
        country_list.append(centre)

df['Country']=country_list
column_list=['Rank', 'Rank Change', 'Centre', 'Country', 'GFCI Rating', 'Rating Change', 'Latitude', 'Longitude']
df=df[column_list]
df.head()

,Rank,Rank Change,Centre,Country,GFCI Rating,Rating Change,Latitude,Longitude
0,1,0,New York,United States,769,21,40.712728,-74.006015
1,2,0,London,United Kingdom,742,31,51.507322,-0.127647
2,3,3,Tokyo,Japan,741,16,35.682839,139.759455
3,4,1,Shanghai,China,740,21,31.225299,121.489050
4,5,1,Singapore,Singapore,738,24,1.357107,103.819499


In [46]:
#First source of data for cities GDP per capita (PPP)

cities_df=pd.read_html('https://en.wikipedia.org/wiki/List_of_cities_by_GDP_(PPP)_per_capita') #Wikipedia source
cities_df=pd.DataFrame(cities_df[0])
cities_df.columns=['MET_ID', 'City', 'GDP Per Capita']
cities_df.head()

,MET_ID,City,GDP Per Capita
0,USA05,San Francisco (Greater),94699
1,LU001,Luxembourg,88312
2,USA14,Seattle,78262
3,USA11,Boston,78183
4,USA01,New York (Greater),74244


In [47]:
#Removing the unnecessary components in the City column to allow joins

city_list=[]
for i in cities_df['City']:
    if '(' in i:
        i=i.split(' (')[0]
#         print(i)
        city_list.append(i)
    else:
#         print(i)
        city_list.append(i)

cities_df['City']=city_list
cities_df.drop_duplicates('City', inplace=True)
cities_df.head()

,MET_ID,City,GDP Per Capita
0,USA05,San Francisco,94699
1,LU001,Luxembourg,88312
2,USA14,Seattle,78262
3,USA11,Boston,78183
4,USA01,New York,74244


In [48]:
econ_df=df.copy()
econ_df=econ_df.merge(cities_df[['City','GDP Per Capita']], how='left', left_on='Centre', right_on='City', )
econ_df.head()

,Rank,Rank Change,Centre,GFCI Rating,Rating Change,Latitude,Longitude,City,GDP Per Capita
0,1,0,New York,769,21,40.712728,-74.006015,New York,74244.0
1,2,0,London,742,31,51.507322,-0.127647,London,58827.0
2,3,3,Tokyo,741,16,35.682839,139.759455,NaN,NaN
3,4,1,Shanghai,740,21,31.225299,121.489050,NaN,NaN
4,5,1,Singapore,738,24,1.357107,103.819499,NaN,NaN


In [49]:
#Second source of data for cities GDP per capita (PPP) 

cities_df1=pd.read_csv('http://www.worldcitiescultureforum.com/assets/city_data/Average_income_per_capita_per_year_%28ppp%29_5112018.csv')
remove_list=['Notes', 'Per capita']
cities_df1.drop(columns=remove_list, inplace=True)

#Removing the $sign in the 'Figure' columns
cities_df1['Figure']=cities_df1['Figure'].str.replace('$', '')
cities_df1['Figure']=cities_df1['Figure'].str.replace(',', '')
cities_df1['Figure']=cities_df1['Figure'].astype('float')
cities_df1.columns=['City1', 'Per Capita', 'Year', 'Source']
cities_df1.head()

,City1,Per Capita,Year,Source
0,Amsterdam,19271.0,2012,Statistics Netherlands/TNO
1,Netherlands,17492.0,2009,Statistics Netherlands /RIO
2,Austin,39103.0,2016,Census Reporter
3,United States,28155.0,2013,US Census Bureau
4,Bogotá,9004.0,2014,Observatorio de Desarrollo Económico de Bogotá...


In [50]:
econ_df=econ_df.merge(cities_df1[['City1', 'Per Capita']], how='left', left_on='Centre', right_on='City1')
econ_df.head()

,Rank,Rank Change,Centre,GFCI Rating,Rating Change,Latitude,Longitude,City,GDP Per Capita,City1,Per Capita
0,1,0,New York,769,21,40.712728,-74.006015,New York,74244.0,New York,34099.0
1,2,0,London,742,31,51.507322,-0.127647,London,58827.0,London,38537.0
2,3,3,Tokyo,741,16,35.682839,139.759455,NaN,NaN,Tokyo,60678.0
3,4,1,Shanghai,740,21,31.225299,121.489050,NaN,NaN,Shanghai,22473.0
4,5,1,Singapore,738,24,1.357107,103.819499,NaN,NaN,Singapore,54224.0


In [51]:
#Filling in the NaN figures in the original GDP per Capita column with the data from the cities_df1 join

gdp_data=[]
for first, second in zip(econ_df['GDP Per Capita'], econ_df['Per Capita']):
    if np.isnan(first):
        gdp_data.append(second)
    else:
        gdp_data.append(first)

econ_df['GDP Per Capita']=gdp_data
econ_df.head()

,Rank,Rank Change,Centre,GFCI Rating,Rating Change,Latitude,Longitude,City,GDP Per Capita,City1,Per Capita
0,1,0,New York,769,21,40.712728,-74.006015,New York,74244.0,New York,34099.0
1,2,0,London,742,31,51.507322,-0.127647,London,58827.0,London,38537.0
2,3,3,Tokyo,741,16,35.682839,139.759455,NaN,60678.0,Tokyo,60678.0
3,4,1,Shanghai,740,21,31.225299,121.489050,NaN,22473.0,Shanghai,22473.0
4,5,1,Singapore,738,24,1.357107,103.819499,NaN,54224.0,Singapore,54224.0


In [52]:
nulls=econ_df['GDP Per Capita'].isnull().sum()
print(f'The count of cities with no GDP per capita data: {nulls}')

The count of cities with no GDP per capita data: 55
